In [1]:
import sys

In [2]:
# if "../src/" not in sys.path:
#     sys.path.insert(0, "../src")

In [3]:
import os
import json
import copy

import plotly.express as px
import numpy as np
import pandas as pd

# from atlas.evaluation import average_substring_accuracy

In [4]:
data_path = "/mimer/NOBACKUP/groups/snic2022-22-1003/APP/qa-retriever/data/atlas/popqa_data/test.jsonl"
experiment_path = "/mimer/NOBACKUP/groups/snic2022-22-1003/APP/qa-retriever/experiments/evaluate-popqa-full-size-base-ctx-5/evaluate-popqa-full-size-base-ctx-5/test-step-0.jsonl"

In [5]:
data = []
with open(data_path, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            data.append(json.loads(line))


output = []
empty_ouput = []
with open(experiment_path, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            out = json.loads(line)
            question = out["query"].replace("question: ", "").replace(" answer: <extra_id_0>", "").strip()

            row = copy.deepcopy([item for item in data if item.get("question") == question])
            if len(row) > 0:
                row = row[0]
                row.update(out)

            if row["generation"].strip() and len(row["generation"]) > 1:
                output.append(row)
            else:
                empty_ouput.append(row)

print(f"Number of generated responses #{len(output)}")
print(f"Number of empty generated responses #{len(empty_ouput)}")

Number of generated responses #13025
Number of empty generated responses #1242


In [6]:
print(f"Unique props: {list(set([r['triplet']['prop'] for r in data]))}")

Unique props: ['screenwriter', 'producer', 'religion', 'sport', 'mother', 'capital of', 'place of birth', 'composer', 'color', 'capital', 'father', 'occupation', 'country', 'author', 'director', 'genre']


In [7]:
output_views = pd.DataFrame([
    {
        "subj": row["triplet"]["subj"],
        "prop": row["triplet"]["prop"],
        "obj": row["triplet"]["obj"],
        "s_pop": int(row["views"]["s_pop"]), 
        "o_pop": int(row["views"]["o_pop"])

    } 
    for row in output
])

output_views.head()

,subj,prop,obj,s_pop,o_pop
0,Kathy Saltzman,occupation,politician,127,25692
1,Leona Detiège,occupation,politician,124,25692
2,Jacob Kraemer,occupation,actor,230,81374
3,Javier Alva Orlandini,occupation,politician,169,25692
4,Sachiko Kokubu,occupation,actor,115,81374


In [8]:
empty_ouput_views = pd.DataFrame([
    {
        "subj": row["triplet"]["subj"],
        "prop": row["triplet"]["prop"],
        "obj": row["triplet"]["obj"],
        "s_pop": int(row["views"]["s_pop"]), 
        "o_pop": int(row["views"]["o_pop"])
    } 
    for row in empty_ouput
])
empty_ouput_views.head()

,subj,prop,obj,s_pop,o_pop
0,Thanasis Pafilis,occupation,politician,285,25692
1,Javad Maroufi,occupation,composer,279,28389
2,Min,occupation,actor,1440,81374
3,Atsushi Kaneko,occupation,mangaka,387,7223
4,Sayaka Ichii,occupation,actor,337,81374


In [9]:
def average_substring_accuracy(predictions, ground_truths_list):
    if isinstance(predictions, str):
        predictions = [predictions]
        ground_truths_list = [ground_truths_list]

    total_accuracy = 0.0
    
    # Ensure both lists have the same length
    assert len(predictions) == len(ground_truths_list), "Length of predictions and ground_truths_list must be the same."
    
    for prediction, ground_truths in zip(predictions, ground_truths_list):
        is_correct = 0
        for ground_truth in ground_truths:
            if prediction in ground_truth:
                is_correct = 1
                break  # Exit loop if a match is found
            
        total_accuracy += is_correct
    
    # Calculate average accuracy across all inputs
    average_accuracy = total_accuracy / len(predictions)
    return average_accuracy

In [12]:
acc_list = []

for i, row in enumerate(output):

    ground_truths = row["answers"]
    prediction = row["generation"].strip()
    prediction = prediction[:-1] if prediction.endswith(".") else prediction

    acc_list.append(average_substring_accuracy(prediction, ground_truths))

print(f"Overal SubAcc: {sum(acc_list) / len(acc_list)}")

Overal SubAcc: 0.08614203454894434


In [13]:
output_by_prop = {}

for i, row in enumerate(output):
    prop = row["triplet"]["prop"]

    if prop not in output_by_prop:
        output_by_prop[prop] = []


    ground_truths = row["answers"]
    prediction = row["generation"].strip()
    prediction = prediction[:-1] if prediction.endswith(".") else prediction

    output_by_prop[prop].append({
        "ground_truths": ground_truths,
        "prediction": prediction,
        "s_pop": int(row["views"]["s_pop"]), 
        "o_pop": int(row["views"]["o_pop"]),
        "acc": average_substring_accuracy(prediction, ground_truths)
    })

print(f"Prop list: {list(output_by_prop.keys())}")

Prop list: ['occupation', 'place of birth', 'genre', 'father', 'country', 'producer', 'director', 'capital of', 'screenwriter', 'composer', 'color', 'religion', 'sport', 'author', 'mother', 'capital']


In [14]:
print("SubAcc over Popularity (Overal):")
print("===" * 20)
for prop in output_by_prop:
    score = [r["acc"] for r in output_by_prop[prop]]
    if len(score) > 0:
        print(f"SubAccuary: {prop} ==> {sum(score)/len(score)}")

SubAcc over Popularity (Overal):
SubAccuary: occupation ==> 0.0311284046692607
SubAccuary: place of birth ==> 0.08043875685557587
SubAccuary: genre ==> 0.03550295857988166
SubAccuary: father ==> 0.15182186234817813
SubAccuary: country ==> 0.22098569157392686
SubAccuary: producer ==> 0.037526059763724806
SubAccuary: director ==> 0.06707946336429309
SubAccuary: capital of ==> 0.37398373983739835
SubAccuary: screenwriter ==> 0.03324937027707808
SubAccuary: composer ==> 0.08351893095768374
SubAccuary: color ==> 0.0
SubAccuary: religion ==> 0.10855263157894737
SubAccuary: sport ==> 0.1062618595825427
SubAccuary: author ==> 0.13026521060842433
SubAccuary: mother ==> 0.11046511627906977
SubAccuary: capital ==> 0.2052313883299799


In [15]:
print("SubAcc over Popularity (100K):")
print("===" * 20)
for prop in output_by_prop:
    score = [r["acc"] for r in output_by_prop[prop] if r["o_pop"] > 100_000]
    if len(score) > 0:
        print(f"SubAccuary: {prop} ==> {sum(score)/len(score)}")

SubAcc over Popularity (100K):
SubAccuary: occupation ==> 0.0
SubAccuary: place of birth ==> 0.0967741935483871
SubAccuary: genre ==> 0.018404907975460124
SubAccuary: father ==> 0.2727272727272727
SubAccuary: country ==> 0.22168284789644013
SubAccuary: producer ==> 0.0297029702970297
SubAccuary: director ==> 0.10576923076923077
SubAccuary: capital of ==> 0.559322033898305
SubAccuary: screenwriter ==> 0.0
SubAccuary: composer ==> 0.05263157894736842
SubAccuary: religion ==> 0.08333333333333333
SubAccuary: sport ==> 0.04071246819338423
SubAccuary: author ==> 0.16071428571428573
SubAccuary: mother ==> 0.2
SubAccuary: capital ==> 0.22807017543859648
